# Cleaning a PostgreSQL Database
![Clean PostgreSQL Database](Project_Image.jpeg)

In this project, you will work with data from a hypothetical Super Store to challenge and enhance your SQL skills in data cleaning. This project will engage you in identifying top categories based on the highest profit margins and detecting missing values, utilizing your comprehensive knowledge of SQL concepts.

## Data Dictionary:

### `orders`:
| Column | Definition | Data type | Comments |
|--------|------------|-----------|----------|
| `row_id`| Unique Record ID | `INTEGER` |
| `order_id` | Identifier for each order in table | `TEXT` | Connects to `order_id` in `returned_orders` table |
| `order_date` | Date when order was placed | `TEXT` |
| `market` | Market order_id belongs to | `TEXT` |
| `region` | Region Customer belongs to | `TEXT` | Connects to `region` in `people` table |
| `product_id` | Identifier of Product bought | `TEXT` | Connects to `product_id` in `products` table |
| `sales` | Total Sales Amount for the Line Item | `DOUBLE PRECISION` |
| `quantity` | Total Quantity for the Line Item | `DOUBLE PRECISION` |
| `discount` | Discount applied for the Line Item | `DOUBLE PRECISION` |
| `profit` | Total Profit earned on the Line Item | `DOUBLE PRECISION` |

### `returned_orders`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `returned`| Yes values for Order / Line Item Returned | `TEXT` |
| `order_id` | Identifier for each order in table | `TEXT` |
| `market` | Market order_id belongs to | `TEXT` |

### `people`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `person`| Name of Salesperson credited with Order | `TEXT` |
| `region` | Region Salesperson in operating in | `TEXT` |

### `products`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `product_id`| Unique Identifier for the Product | `TEXT` |
| `category` | Category Product belongs to | `TEXT` |
| `sub_category` | Sub Category Product belongs to | `TEXT` |
| `product_name` | Detailed Name of the Product | `TEXT` |

As you can see in the Data Dictionary above, date fields have been written to the `orders` table as `TEXT` and numeric fields like sales, profit, etc. have been written to the `orders` table as `Double Precision`. You will need to take care of these types in some of the queries. This project is an excellent opportunity to apply your SQL skills in a practical setting and gain valuable experience in data cleaning and analysis. Good luck, and happy querying!

In [13]:
-- Use a common table expression (CTE) to aggregate total sales and profit for each product.
WITH product_aggregates AS (
    SELECT 
        p.category,                                 -- Retrieves the product category from the products table.
        p.product_name,                             -- Retrieves the product name.
        ROUND(SUM(o.sales)::numeric, 2) AS product_total_sales,   -- Sums the sales for each product, casts the result to numeric, and rounds it to two decimal places.
        ROUND(SUM(o.profit)::numeric, 2) AS product_total_profit  -- Sums the profit for each product, casts the result to numeric, and rounds it to two decimal places.
    FROM orders o
    JOIN products p 
        ON o.product_id = p.product_id              -- Joins orders and products tables based on product_id.
    GROUP BY p.category, p.product_name            -- Groups the data by category and product name to perform the aggregations.
),

-- Use another CTE to rank products within each category by total sales in descending order.
ranked_products AS (
    SELECT
        category,
        product_name,
        product_total_sales,
        product_total_profit,
        RANK() OVER (PARTITION BY category ORDER BY product_total_sales DESC) AS product_rank
        -- RANK() calculates the rank of each product within its category, ordering by product_total_sales in descending order.
    FROM product_aggregates
)

-- Select the top 5 products per category.
SELECT 
    category,
    product_name,
    product_total_sales,
    product_total_profit,
    product_rank
FROM ranked_products
WHERE product_rank <= 5                    -- Filters the results to include only the top 5 ranked products per category.
ORDER BY category ASC, product_total_sales DESC;
-- Orders the final output by category (alphabetically) and then by product_total_sales (highest first within each category).

,category,product_name,product_total_sales,product_total_profit,product_rank
0,Furniture,"Hon Executive Leather Armchair, Adjustable",58193.48,5997.25,1
1,Furniture,"Office Star Executive Leather Armchair, Adjust...",51449.80,4925.80,2
2,Furniture,"Harbour Creations Executive Leather Armchair, ...",50121.52,10427.33,3
3,Furniture,"SAFCO Executive Leather Armchair, Black",41923.53,7154.28,4
4,Furniture,"Novimex Executive Leather Armchair, Adjustable",40585.13,5562.35,5
5,Office Supplies,"Eldon File Cart, Single Width",39873.23,5571.26,1
6,Office Supplies,"Hoover Stove, White",32842.60,-2180.63,2
7,Office Supplies,"Hoover Stove, Red",32644.13,11651.68,3
8,Office Supplies,"Rogers File Cart, Single Width",29558.82,2368.82,4
9,Office Supplies,"Smead Lockers, Industrial",28991.66,3630.44,5


In [14]:
-- impute_missing_values
-- First, compute the average unit price for each product_id, discount, market, and region.
-- Unit price is defined as sales divided by quantity (from rows where quantity is known and > 0).
WITH unit_prices AS (
    SELECT
        product_id,                              -- Product identifier
        discount,                                -- Discount applied
        market,                                  -- Market identifier
        region,                                  -- Region of the customer/salesperson
        AVG(sales / quantity) AS unit_price       -- Average unit price computed from known quantities
    FROM orders
    WHERE quantity IS NOT NULL AND quantity > 0  -- Use only orders with valid quantity
    GROUP BY product_id, discount, market, region  -- Group by product and pricing factors
)

-- Next, join the orders with missing quantity to the computed unit_prices.
SELECT
    o.product_id,                                      -- Product identifier from orders
    o.discount,                                        -- Discount applied in the order
    o.market,                                          -- Market identifier in the order
    o.region,                                          -- Region in the order
    o.sales,                                           -- Total sales amount from the order
    o.quantity,                                        -- Original (missing) quantity value
    -- Calculate the missing quantity by dividing sales by the unit price and round to 0 decimals.
    ROUND(o.sales / up.unit_price) AS calculated_quantity  
FROM orders o
JOIN unit_prices up 
    ON o.product_id = up.product_id                 -- Join on product_id
   AND o.discount = up.discount                     -- Join on discount
   AND o.market = up.market                         -- Join on market
   AND o.region = up.region                         -- Join on region
WHERE o.quantity IS NULL;                           -- Only process orders with missing quantity


,product_id,discount,market,region,sales,quantity,calculated_quantity
0,TEC-STA-10003330,0.00,Africa,Africa,506.640,NaN,2
1,FUR-ADV-10000571,0.00,EMEA,EMEA,438.960,NaN,4
2,FUR-BO-10001337,0.15,US,West,308.499,NaN,3
3,TEC-STA-10004542,0.00,Africa,Africa,160.320,NaN,4
4,FUR-ADV-10004395,0.00,EMEA,EMEA,84.120,NaN,2


In [21]:
-- Select key metrics to analyze customer purchase behavior and sales trends.
SELECT
    EXTRACT(YEAR FROM TO_DATE(order_date, 'YYYY-MM-DD')) AS order_year,  
        -- Extracts the year from the order_date (assumes order_date is in 'YYYY-MM-DD' format).
    region,                        
        -- Returns the region from the orders table.
    market,                        
        -- Returns the market from the orders table.
    COUNT(DISTINCT order_id) AS num_orders,  
        -- Counts the number of distinct orders for that year, region, and market.
    COUNT(*) AS total_line_items,  
        -- Counts the total number of line items (each row represents an order line).
    SUM(sales) AS total_sales,     
        -- Sums up the total sales amount for those orders.
    ROUND(AVG(sales)::numeric, 2) AS avg_sales_per_order,  
        -- Calculates the average sales per order and rounds it to two decimal places.
    SUM(quantity) AS total_quantity  
        -- Sums up the total quantity sold for those orders.
FROM orders

GROUP BY order_year, region, market  
        -- Groups the data by extracted order year, region, and market.
ORDER BY order_year DESC, total_sales DESC; 
        -- Orders the results by year, then region, then market in ascending order.

,order_year,region,market,num_orders,total_line_items,total_sales,avg_sales_per_order,total_quantity
0,2014,Central,EU,843,2008,597898.94550,297.76,7516
1,2014,Oceania,APAC,625,1193,362429.88600,303.80,4334
2,2014,Southeast Asia,APAC,534,1112,323068.21230,290.53,4299
3,2014,EMEA,EMEA,888,1811,301685.94300,166.59,4155
4,2014,Africa,Africa,813,1624,283036.44000,174.28,3741
...,...,...,...,...,...,...,...,...
67,2011,North,EU,181,334,101576.82900,304.12,1194
68,2011,South,LATAM,231,448,93798.80052,209.37,1697
69,2011,South,EU,172,357,86397.55200,242.01,1291
70,2011,Caribbean,LATAM,153,289,57043.42896,197.38,1044


In [43]:
import plotly.express as px

# Sort the Customer_Order dataframe by order_year (descending) and total_sales (descending)
Customer_Order_sorted = Customer_Order.sort_values(by=['order_year', 'total_sales'], ascending=[False, False])
# Now the data is ordered so that the most recent years with the highest sales come first.

# Create a bar chart with Plotly Express:
# - x-axis: order_year
# - y-axis: total_sales
# - color: region (to add another dimension to the visualization)
# - The title describes the analysis.
fig = px.bar(
    Customer_Order_sorted,
    x='order_year',
    y='total_sales',
    color='region',
    title='Customer Order Analysis: Total Sales by Year (Descending Order)',
    labels={'order_year': 'Order Year', 'total_sales': 'Total Sales'}
)

# Display the plot
fig.show()

In [27]:
-- First, aggregate order data by month. We assume order_date is stored as TEXT in 'YYYY-MM-DD' format.
WITH monthly_aggregates AS (
    SELECT
        -- Truncate the order_date to the first day of the month to group by month
        DATE_TRUNC('month', TO_DATE(order_date, 'YYYY-MM-DD'))::date AS order_month,
        -- Sum total sales for each month
        SUM(sales) AS total_sales,
        -- Sum total profit for each month
        SUM(profit) AS total_profit,
        -- Count the number of distinct orders (as a proxy for order volume)
        COUNT(DISTINCT order_id) AS order_volume
    FROM orders
    GROUP BY order_month
)

-- Now, use window functions on the monthly aggregates to calculate moving averages and growth rates
SELECT
    order_month,                              -- The month of the orders
    total_sales,                              -- Total sales for that month
    total_profit,                             -- Total profit for that month
    order_volume,                             -- Number of orders in that month
    -- Calculate a 3-month moving average of total sales (current month and two preceding months)
    ROUND(AVG(total_sales) OVER (
        ORDER BY order_month 
        ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
    )::numeric, 2) AS moving_avg_sales,
    -- Use LAG() to get the total sales from the previous month
    LAG(total_sales) OVER (ORDER BY order_month) AS previous_month_sales,
    -- Calculate the percentage growth in sales from the previous month, rounded to 2 decimals.
    ROUND(
        CASE 
            WHEN LAG(total_sales) OVER (ORDER BY order_month) IS NULL THEN NULL
            WHEN LAG(total_sales) OVER (ORDER BY order_month) = 0 THEN NULL
            ELSE ((total_sales - LAG(total_sales) OVER (ORDER BY order_month)) * 100.0 
                  / LAG(total_sales) OVER (ORDER BY order_month))
        END::numeric, 
        2
    ) AS sales_growth_percent
FROM monthly_aggregates
ORDER BY order_month DESC, total_sales DESC;

,order_month,total_sales,total_profit,order_volume,moving_avg_sales,previous_month_sales,sales_growth_percent
0,2014-12-01 00:00:00+00:00,503143.69348,46916.52068,1093,493729.78,555279.02700,-9.39
1,2014-11-01 00:00:00+00:00,555279.02700,62856.58790,1077,486400.97,422766.62916,31.34
2,2014-10-01 00:00:00+00:00,422766.62916,58209.83476,810,453514.61,481157.24370,-12.14
3,2014-09-01 00:00:00+00:00,481157.24370,67979.45110,1017,398827.62,456619.94236,5.37
4,2014-08-01 00:00:00+00:00,456619.94236,53542.89496,843,372379.90,258705.68048,76.50
5,2014-07-01 00:00:00+00:00,258705.68048,28035.87258,540,316306.93,401814.06310,-35.62
6,2014-06-01 00:00:00+00:00,401814.06310,43778.60280,899,310995.66,288401.04614,39.32
7,2014-05-01 00:00:00+00:00,288401.04614,33953.55774,667,264757.89,242771.86130,18.80
8,2014-04-01 00:00:00+00:00,242771.86130,23782.30120,523,230236.66,263100.77262,-7.73
9,2014-03-01 00:00:00+00:00,263100.77262,37357.26052,530,229735.56,184837.35556,42.34


In [31]:
-- First, aggregate overall order data per product from the orders table.
WITH product_orders AS (
    SELECT
        o.product_id,                              -- Product identifier from orders table
        COUNT(*) AS total_orders,                  -- Total orders for this product
        SUM(o.sales) AS total_sales,               -- Total sales for this product
        SUM(o.profit) AS total_profit              -- Total profit for this product
    FROM orders o
    GROUP BY o.product_id                         -- Group by product_id to aggregate per product
),

-- Next, aggregate returned orders data by joining orders with returned_orders.
product_returns AS (
    SELECT
        o.product_id,                              -- Product identifier
        COUNT(*) AS returned_orders,               -- Number of returned orders for this product
        SUM(o.sales) AS returned_sales,            -- Total sales for returned orders
        SUM(o.profit) AS returned_profit           -- Total profit for returned orders
    FROM orders o
    INNER JOIN returned_orders r                  -- Join orders with returned_orders using order_id
        ON o.order_id = r.order_id
    GROUP BY o.product_id                         -- Group by product_id to aggregate per product
)

-- Finally, join the aggregated order data with the products table to get product details.
SELECT
    p.product_id,                                          -- Product identifier from products table
    p.product_name,                                        -- Product name from products table
    po.total_orders,                                       -- Total number of orders for the product
    pr.returned_orders,                                    -- Total number of returned orders for the product
    ROUND((pr.returned_orders::numeric / po.total_orders) * 100, 2) AS return_rate_percentage,
                                                         -- Calculate and round return rate as percentage
    po.total_sales,                                        -- Total sales for the product
    pr.returned_sales,                                     -- Sales from returned orders
    po.total_profit,                                       -- Total profit for the product
    pr.returned_profit                                     -- Profit lost from returned orders
FROM products p
JOIN product_orders po ON p.product_id = po.product_id    -- Join aggregated orders to products
JOIN product_returns pr ON p.product_id = pr.product_id   -- Join aggregated returns to products 
ORDER BY returned_orders DESC, returned_profit DESC;                     -- Order by return rate (highest first)


,product_id,product_name,total_orders,returned_orders,return_rate_percentage,total_sales,returned_sales,total_profit,returned_profit
0,FUR-CH-10003354,"Harbour Creations Swivel Stool, Red",28,5,17.86,8184.5901,1946.8020,1128.0801,367.0920
1,TEC-PH-10002564,"Motorola Signal Booster, Full Size",15,5,33.33,3570.9490,1522.9290,335.0980,120.7920
2,OFF-EN-10003529,"Ames Peel and Seal, Recycled",16,5,31.25,877.2120,229.5280,57.9720,-1.0320
3,OFF-PA-10001970,Xerox 1908,19,4,21.05,2167.3480,603.7200,1006.4352,250.6218
4,OFF-SU-10003473,"Elite Trimmer, Steel",6,4,66.67,749.1168,642.7455,206.5968,198.8655
...,...,...,...,...,...,...,...,...,...
2479,TEC-MA-10001695,Zebra GK420t Direct Thermal/Thermal Transfer P...,1,1,100.00,703.7100,703.7100,-938.2800,-938.2800
2480,TEC-MA-10000864,Cisco 9971 IP Video Phone Charcoal,2,1,50.00,4268.0000,1188.0000,466.4000,-950.4000
2481,FUR-TA-10003003,"Lesro Wood Table, Rectangular",2,1,50.00,1697.2500,1046.2500,-1134.6000,-976.5000
2482,FUR-TA-10000519,"Bevis Computer Table, Fully Assembled",1,1,100.00,692.3280,692.3280,-1315.5520,-1315.5520


In [36]:
-- Step 1: Aggregate sales and profit by category and sub-category
WITH category_profitability AS (
    SELECT
        p.category,                                    -- Product category (e.g., Technology, Furniture)
        p.sub_category,                                -- Product sub-category (e.g., Chairs, Phones)
        SUM(o.sales) AS total_sales,                   -- Total sales amount for the sub-category
        SUM(o.profit) AS total_profit,                 -- Total profit earned from that sub-category
        -- Calculate profit margin: profit divided by sales, cast to numeric, rounded to 2 decimal places
        ROUND((SUM(o.profit) / NULLIF(SUM(o.sales), 0))::numeric, 2) AS profit_margin
    FROM orders o
    JOIN products p ON o.product_id = p.product_id     -- Join orders with product info to access category data
    GROUP BY p.category, p.sub_category                -- Group the data by category and sub-category
),

-- Step 2: Rank sub-categories by profit margin within each category
ranked_profitability AS (
    SELECT
        category,
        sub_category,
        total_sales,
        total_profit,
        profit_margin,
        -- Assign rank within each category based on highest to lowest profit margin
        RANK() OVER (
            PARTITION BY category 
            ORDER BY profit_margin DESC
        ) AS margin_rank
    FROM category_profitability
)

-- Step 3: Final result selection and ordering
SELECT 
    category,                       -- Product category
    sub_category,                   -- Product sub-category
    total_sales,                    -- Total sales in dollars
    total_profit,                   -- Total profit in dollars
    profit_margin,                  -- Profit as a percentage of sales
    margin_rank                     -- Rank within the category by profit margin
FROM ranked_profitability
ORDER BY category ASC, profit_margin DESC;  -- Order first by category, then by highest profit margin


,category,sub_category,total_sales,total_profit,profit_margin,margin_rank
0,Furniture,Furnishings,3.855783e+05,46967.42550,0.12,1
1,Furniture,Bookcases,1.466572e+06,161924.41950,0.11,2
2,Furniture,Chairs,1.501682e+06,140396.26750,0.09,3
3,Furniture,Tables,7.570419e+05,-64083.38870,-0.08,4
4,Office Supplies,Paper,2.442917e+05,59207.68270,0.24,1
5,Office Supplies,Labels,7.338437e+04,15008.85600,0.20,2
6,Office Supplies,Envelopes,1.709043e+05,29601.11630,0.17,3
7,Office Supplies,Binders,4.619312e+05,72451.50200,0.16,4
8,Office Supplies,Art,3.720920e+05,57953.91090,0.16,4
9,Office Supplies,Fasteners,8.324232e+04,11525.42410,0.14,6


In [37]:
-- Step 1: Join orders with the people table using the region column
SELECT
    o.region,                                     -- Region of the customer
    o.market,                                     -- Market where the order was placed
    p.person AS salesperson,                      -- Salesperson responsible, from the people table
    SUM(o.sales) AS total_sales,                  -- Total sales amount in this region-market-person group
    SUM(o.profit) AS total_profit,                -- Total profit earned
    -- Calculate average profit margin (profit / sales), rounded to 2 decimals
    ROUND((SUM(o.profit) / NULLIF(SUM(o.sales), 0))::numeric, 2) AS profit_margin
FROM orders o
JOIN people p ON o.region = p.region              -- Join on region (associates salesperson with order region)
GROUP BY o.region, o.market, p.person             -- Group by region, market, and salesperson
ORDER BY total_sales DESC;                        -- Sort by highest performing areas in terms of total sales


,region,market,salesperson,total_sales,total_profit,profit_margin
0,Central,EU,Anna Andreadi,1.720553e+06,215534.06850,0.13
1,Oceania,APAC,Anthony Jacobs,1.100185e+06,120089.11200,0.11
2,Southeast Asia,APAC,Alejandro Ballentine,8.844232e+05,17852.32900,0.02
3,North Asia,APAC,Shirley Daniels,8.483098e+05,165578.42100,0.20
4,EMEA,EMEA,Larry Hughes,8.061613e+05,43897.97100,0.05
5,Africa,Africa,Deborah Brumfield,7.837732e+05,88871.63100,0.11
6,Central Asia,APAC,Nora Preis,7.528266e+05,132480.18700,0.18
7,West,US,Matt Collister,7.254578e+05,108418.44890,0.15
8,East,US,Kelly Williams,6.787812e+05,91522.78000,0.13
9,North,EU,Jack Lebron,6.255750e+05,91779.85500,0.15


In [39]:
-- Step 1: Join orders with people and calculate total sales and profit per salesperson
SELECT
    p.person AS salesperson,                         -- Salesperson’s name
    SUM(o.sales) AS total_sales,                     -- Total sales amount handled by this salesperson
    SUM(o.profit) AS total_profit,                   -- Total profit generated by this salesperson
    -- Calculate profit margin as (profit / sales), rounded to 2 decimal places
    ROUND((SUM(o.profit) / NULLIF(SUM(o.sales), 0))::numeric, 2) AS profit_margin
FROM orders o
JOIN people p ON o.region = p.region                 -- Join using region (associates salesperson to order)
GROUP BY p.person                                    -- Group results by salesperson
ORDER BY total_sales DESC;                           -- Sort by highest total sales


,salesperson,total_sales,total_profit,profit_margin
0,Anna Andreadi,2.822303e+06,311403.98164,0.11
1,Chuck Magee,1.600907e+06,140355.76618,0.09
2,Jack Lebron,1.248166e+06,194597.95252,0.16
3,Anthony Jacobs,1.100185e+06,120089.11200,0.11
4,Alejandro Ballentine,8.844232e+05,17852.32900,0.02
5,Shirley Daniels,8.483098e+05,165578.42100,0.20
6,Larry Hughes,8.061613e+05,43897.97100,0.05
7,Deborah Brumfield,7.837732e+05,88871.63100,0.11
8,Nora Preis,7.528266e+05,132480.18700,0.18
9,Matt Collister,7.254578e+05,108418.44890,0.15


In [42]:
import plotly.express as px

# Round the profit margin to 4 decimal places
salesperson_df['profit_margin'] = salesperson_df['profit_margin'].round(4)

# Create a horizontal bar chart
fig = px.bar(
    salesperson_df,
    x='total_sales',               # Total sales on the x-axis
    y='salesperson',               # Salesperson names on the y-axis
    orientation='h',               # Horizontal bars
    color='total_sales',           # Color bars based on total sales
    text='total_sales',            # Show sales values on bars
    hover_data={
        'total_profit': True,      # Show total profit in hover
        'profit_margin': True,     # Show profit margin (rounded)
        'total_sales': ':.2f',     # Format sales nicely in hover
        'salesperson': False       # Already on y-axis, no need to repeat in hover
    },
    title='Total Sales by Salesperson'
)

# Customize layout
fig.update_layout(
    xaxis_title='Total Sales ($)',
    yaxis_title='Salesperson',
    coloraxis_showscale=False,  # Hide color scale legend
    template='plotly_white'
)

# Show the figure
fig.show()


In [44]:
SELECT * 
FROM orders;

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,city,state,country,postal_code,market,region,product_id,sales,quantity,discount,profit,shipping_cost,order_priority
0,957,MX-2014-105921,2014-05-28,2014-06-03,Standard Class,ZC-21910,Zuschuss Carroll,Consumer,San Salvador,San Salvador,El Salvador,NaN,LATAM,Central,TEC-AC-10004626,342.080,2.0,0.00,0.0000,21.713,Medium
1,24359,ID-2013-61442,2013-01-15,2013-01-21,Standard Class,JB-16000,Joy Bell-,Consumer,Manila,National Capital,Philippines,NaN,APAC,Southeast Asia,OFF-BI-10001400,122.400,5.0,0.15,0.0000,21.710,Low
2,32298,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,United States,10024.0,US,East,TEC-AC-10003033,2309.650,7.0,0.00,762.1845,933.570,Critical
3,26341,IN-2013-77878,2013-02-05,2013-02-07,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,Australia,NaN,APAC,Oceania,FUR-CH-10003950,3709.395,9.0,0.10,-288.7650,923.630,Critical
4,25330,IN-2013-71249,2013-10-17,2013-10-18,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,Australia,NaN,APAC,Oceania,TEC-PH-10004664,5175.171,9.0,0.10,919.9710,915.490,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51285,29002,IN-2014-62366,2014-06-19,2014-06-19,Same Day,KE-16420,Katrina Edelman,Corporate,Kure,Hiroshima,Japan,NaN,APAC,North Asia,OFF-FA-10000746,65.100,5.0,0.00,4.5000,0.010,Medium
51286,35398,US-2014-102288,2014-06-20,2014-06-24,Standard Class,ZC-21910,Zuschuss Carroll,Consumer,Houston,Texas,United States,77095.0,US,Central,OFF-AP-10002906,0.444,1.0,0.80,-1.1100,0.010,Medium
51287,40470,US-2013-155768,2013-12-02,2013-12-02,Same Day,LB-16795,Laurel Beltran,Home Office,Oxnard,California,United States,93030.0,US,West,OFF-EN-10001219,22.920,3.0,0.00,11.2308,0.010,High
51288,9596,MX-2012-140767,2012-02-18,2012-02-22,Standard Class,RB-19795,Ross Baird,Home Office,Valinhos,São Paulo,Brazil,NaN,LATAM,South,OFF-BI-10000806,13.440,2.0,0.00,2.4000,0.003,Medium


In [45]:
# Check for missing values and ensure numeric types
print(Orders_df[['discount', 'sales', 'quantity', 'profit']].info())
print("\nMissing values:\n", Orders_df[['discount', 'sales', 'quantity', 'profit']].isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   discount  51290 non-null  float64
 1   sales     51290 non-null  float64
 2   quantity  51285 non-null  float64
 3   profit    51290 non-null  float64
dtypes: float64(4)
memory usage: 1.6 MB
None

Missing values:
 discount    0
sales       0
quantity    5
profit      0
dtype: int64


In [49]:
# Drop rows where quantity is missing
# exclude rows where quantity is missing so the analysis remains accurate.
clean_orders_df = Orders_df.dropna(subset=['quantity'])

# Calculate correlation matrix
correlation_matrix = clean_orders_df[['discount', 'sales', 'quantity', 'profit']].corr()

print("🔍 Correlation Matrix:")
print(correlation_matrix)


🔍 Correlation Matrix:
          discount     sales  quantity    profit
discount  1.000000 -0.086718 -0.019886 -0.316477
sales    -0.086718  1.000000  0.313584  0.484914
quantity -0.019886  0.313584  1.000000  0.104379
profit   -0.316477  0.484914  0.104379  1.000000
